make a new file that loads the h5 & csv with labels. drop rows in the csv that don't have a label or have DELETE as their label. drop rows from h5 that aren't in csv. output smaller h5 that's useable

In [1]:
import pandas as pd
import os
from pathlib import Path
import librosa
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import h5py
from tqdm import tqdm

In [2]:
# get file paths
FOLDER_PATH = "/Users/cheoso/Documents/samples/Packs/"
file_paths = []
total = 0

for root, dirs, files in os.walk(FOLDER_PATH):
    for file in files:
        path = Path(root) / file
        path_lower = str(path).lower()
        if path_lower.endswith(('.wav', '.aif', '.aiff')):
          file_paths.append(str(path))

          total += path.stat().st_size

print(f"Total of {total} bytes.")

Total of 27801282123 bytes.


In [3]:
failed_files = []


def extract_audio_and_metadata(index, file):
    """Load an audio file and extract its waveform, sample rate, and label."""
    try:
        audio, sr = librosa.load(file, sr=None)  # Load without resampling
        file = str(file)
        file_name = file.split('/')[-1]
        return [index, sr, audio], [index, file, file_name]
    except Exception as e:
        failed_files.append(file)
        return None

# Use parallel processing to load audio files
def get_audio_and_metadata(files):
    audio_results = []
    meta_results = []

    with ThreadPoolExecutor() as executor:
        results = executor.map(lambda p: extract_audio_and_metadata(*p), enumerate(files))
        
        for res in results:
            if res is not None:
                audio_data, meta_data = res
                audio_results.append(audio_data)
                meta_results.append(meta_data)
    
    return audio_results, meta_results


audioResults, metadataResults = get_audio_and_metadata(file_paths)

/var/folders/h0/pzm07l6x5n7181rc09xppl940000gn/T/ipykernel_17450/3117495476.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file, sr=None)  # Load without resampling
/Users/cheoso/ai_projects/load_audio/venv/lib/python3.13/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [4]:
failed_len = len(failed_files)
print(failed_len)

638


In [5]:
print(len(audioResults))
print(len(metadataResults))

17698
17698


In [6]:
# Convert results to DataFrame
df_ar = pd.DataFrame(audioResults, columns=["id", "sample_rate", "waveform"])
df_md = pd.DataFrame(metadataResults, columns=["id", "file_path", "file_name"])

In [7]:
df_ar['id'].dtype

dtype('int64')

In [8]:
dt = h5py.string_dtype(encoding='utf-8')


with h5py.File("audio_data.h5", "w") as f:
    meta_data = f.create_group("meta_data")
    meta_data.create_dataset('index', data=np.array(df_md['id'], dtype='int'))
    meta_data.create_dataset('path', data=np.array(df_md['file_path'], dtype=dt))
    meta_data.create_dataset('name', data=np.array(df_md['file_name'], dtype=dt))

    audio_data = f.create_group('audio_data')

    def save_sample(row):
        sample_group = audio_data.create_group(str(row['id']))
        sample_group.create_dataset('waveform', data=row['waveform'], compression='gzip')
        sample_group.create_dataset('sample_rate', data=row['sample_rate'])

    # Use ThreadPoolExecutor for parallelism
    with ThreadPoolExecutor(max_workers=8) as executor:
        list(tqdm(executor.map(save_sample, [row for _, row in df_ar.iterrows()]), total=len(df_ar)))
    

100%|██████████| 17698/17698 [13:42<00:00, 21.51it/s] 


In [9]:
df_md.to_csv("audio_metadata.csv", header=True, index=False)

In [10]:
df_md.head(5)

,id,file_path,file_name
0,0,/Users/cheoso/Documents/samples/Packs/free 808...,808 - old town road.wav
1,1,/Users/cheoso/Documents/samples/Packs/free 808...,808 - new slaves.wav
2,2,/Users/cheoso/Documents/samples/Packs/free 808...,808 - praise the lord 2.wav
3,3,/Users/cheoso/Documents/samples/Packs/free 808...,808 - 7 rings.wav
4,4,/Users/cheoso/Documents/samples/Packs/free 808...,808 - bank account.wav
